In [1]:
import os

In [4]:
%pwd

'/Users/tanmay/Documents/Text Summarizer'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class data_ingestion_config:
    root_dir : Path
    source_url : str
    local_data_file: Path
    unzip_dir: Path


In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_dirs

In [9]:
class configuration_manager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_dirs([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> data_ingestion_config:
        config = self.config.data_ingestion
        create_dirs([config.root_dir])
        dic = data_ingestion_config(
            root_dir= config.root_dir,
            source_url=config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir=config.unzip_dir
        ) 

        return dic

In [10]:
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size



In [12]:
class DataIngestion:
    def __init__(self,config:data_ingestion_config):
        self.config = config

    def download(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url=self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with {headers} headers")
        else:
            logger.info(f"file already exits of size {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as file:
            file.extractall(unzip_path)

In [13]:
try:
    config = configuration_manager()
    data_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_config) 
    data_ingestion.download()
    data_ingestion.extract_zip()

except Exception as e:
    raise e

[2024-06-07 15:09:08,359 : INFO : common : yaml file: config/config.yaml is loaded successfully]
[2024-06-07 15:09:08,364 : INFO : common : yaml file: params.yaml is loaded successfully]
[2024-06-07 15:09:08,366 : INFO : common : Created a directory at : artifacts]
[2024-06-07 15:09:08,366 : INFO : common : Created a directory at : artifacts/data_ingestion]
[2024-06-07 15:09:11,417 : INFO : 3513749377 : artifacts/data_ingestion/data.zip downloaded with Connection: close
Content-Length: 4176912
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3a33dbed522ef282530af1a2b5474287ea29e9ee0cb3cff5babaf3220425c4dc"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 26C2:1B5728:38CE8C:41903C:6662D53D
Accept-Ranges: bytes
Date: Fri, 07 Jun 2024 09:39:10 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4736-BOM
